# PostgreSQL → pandas 取り込みノート（NeuralForecast テーブル一括, v2）

In [1]:

# %pip install sqlalchemy psycopg2-binary pandas
import os, pandas as pd
TARGET_SCHEMA = os.getenv("TARGET_SCHEMA", "public")
TARGET_TABLES = [
    "nf_model_profile","nf_dataset_profile","nf_training_state","nf_weight_statistics",
    "nf_model_complexity","nf_model_diagnosis","nf_parameter_sensitivity","nf_optimization_suggestions",
    "nf_runs","nf_models","nf_model","nf_hparams","nf_ckpt","nf_config","nf_pkl",
]
MAX_ROWS, HEAD_N = None, 10


In [2]:

DATABASE_URL = os.getenv("DATABASE_URL")
if not DATABASE_URL:
    host = os.getenv("PGHOST", "127.0.0.1")
    port = os.getenv("PGPORT", "5432")
    db   = os.getenv("PGDATABASE", "postgres")
    user = os.getenv("PGUSER", "postgres")
    pw   = os.getenv("PGPASSWORD", "z")
    DATABASE_URL = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{db}"
print("[info] DATABASE_URL set (masked)")

[info] DATABASE_URL set (masked)


In [3]:

from sqlalchemy import create_engine, text
engine = create_engine(DATABASE_URL, pool_pre_ping=True)
with engine.connect() as cx:
    cx.execute(text("SELECT 1"))
print("[ok] Connected via SQLAlchemy")

def q(sql: str, params=None) -> pd.DataFrame:
    with engine.begin() as cx:
        return pd.read_sql(text(sql), cx, params=params or {})


[ok] Connected via SQLAlchemy


In [6]:
from sqlalchemy import text, bindparam

sql = text("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = :schema
  AND table_name IN :names
ORDER BY table_name
""").bindparams(bindparam("names", expanding=True))

with engine.begin() as cx:
    exists_df = pd.read_sql(sql, cx, params={"schema": TARGET_SCHEMA, "names": TARGET_TABLES})

existing_tables = exists_df["table_name"].tolist()
missing_tables  = [t for t in TARGET_TABLES if t not in existing_tables]
print("existing:", existing_tables)
print("missing:", missing_tables)


existing: ['nf_ckpt', 'nf_config', 'nf_dataset_profile', 'nf_hparams', 'nf_model', 'nf_model_complexity', 'nf_model_diagnosis', 'nf_model_profile', 'nf_models', 'nf_optimization_suggestions', 'nf_parameter_sensitivity', 'nf_pkl', 'nf_runs', 'nf_training_state', 'nf_weight_statistics']
missing: []


In [7]:

dfs = {}
for t in existing_tables:
    lim = f" LIMIT {int(MAX_ROWS)}" if MAX_ROWS else ""
    df = q(f'SELECT * FROM "{TARGET_SCHEMA}"."{t}"{lim}')
    dfs[t] = df
    globals()[f"df_{t}"] = df
    print(t, len(df), "rows")

# summary
import pandas as pd
summary_df = pd.DataFrame([(t, len(df), len(df.columns)) for t, df in dfs.items()],
                          columns=["table","rows","cols"]).sort_values("table").reset_index(drop=True)
display(summary_df)
if existing_tables:
    t0 = existing_tables[0]
    print(f"=== {t0} head({HEAD_N}) ===")
    display(dfs[t0].head(HEAD_N))


nf_ckpt 1 rows
nf_config 1 rows
nf_dataset_profile 0 rows
nf_hparams 1 rows
nf_model 1 rows
nf_model_complexity 0 rows
nf_model_diagnosis 0 rows
nf_model_profile 0 rows
nf_models 58 rows
nf_optimization_suggestions 0 rows
nf_parameter_sensitivity 0 rows
nf_pkl 1 rows
nf_runs 7 rows
nf_training_state 0 rows
nf_weight_statistics 0 rows


,table,rows,cols
0,nf_ckpt,1,4
1,nf_config,1,16
2,nf_dataset_profile,0,12
3,nf_hparams,1,46
4,nf_model,1,62
5,nf_model_complexity,0,7
6,nf_model_diagnosis,0,7
7,nf_model_profile,0,11
8,nf_models,58,10
9,nf_optimization_suggestions,0,8


=== nf_ckpt head(10) ===


,id,dir_data_long_parent_dir_n_features_loto_bingo5_model_autokan_n,sources,created_at
0,1,"""alias"": None\n...",{'dir_data_long_parent_dir_n_features_loto_bin...,2025-11-11 11:04:02.485786
